In [1]:
from pyspark import SparkContext
sc = SparkContext()


In [2]:
import sys
print(sys.path)

['', '/tmp/spark-81372894-6395-4b37-82cd-e30b039cdfa2/userFiles-0dc111b1-162d-4bed-bc47-bf5705689ba9/org.mongodb_mongo-java-driver-3.2.2.jar', '/tmp/spark-81372894-6395-4b37-82cd-e30b039cdfa2/userFiles-0dc111b1-162d-4bed-bc47-bf5705689ba9/org.mongodb.spark_mongo-spark-connector_2.11-2.0.0.jar', '/tmp/spark-81372894-6395-4b37-82cd-e30b039cdfa2/userFiles-0dc111b1-162d-4bed-bc47-bf5705689ba9', '/usr/local/spark/python', '/home/phejimlin/Documents/dcard_spark/SPARK_HOME/python/lib/py4j-0.10.4-src.zip', '/home/phejimlin/anaconda3/envs/spark/lib/python35.zip', '/home/phejimlin/anaconda3/envs/spark/lib/python3.5', '/home/phejimlin/anaconda3/envs/spark/lib/python3.5/plat-linux', '/home/phejimlin/anaconda3/envs/spark/lib/python3.5/lib-dynload', '/home/phejimlin/.local/lib/python3.5/site-packages', '/home/phejimlin/anaconda3/envs/spark/lib/python3.5/site-packages', '/home/phejimlin/anaconda3/envs/spark/lib/python3.5/site-packages/environment_kernels-1.0-py3.5.egg', '/home/phejimlin/anaconda3/env

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
textFile = sc.textFile("README.md")

In [5]:
textFile.count()
textFile.first()

'# Apache Spark'

In [6]:
linesWithSpark = textFile.filter(lambda line: "Spark" in line)
textFile.filter(lambda line: "Spark" in line).count()  # How many lines contain "Spark"?

20

In [7]:
textFile.map(lambda line: len(line.split())).reduce(lambda a, b: a if (a > b) else b)

22

In [8]:
from pyspark.ml.clustering import LDA

# Loads data.
dataset = spark.read.format("libsvm").load("/usr/local/spark/data/mllib/sample_lda_libsvm_data.txt")

# Trains a LDA model.
lda = LDA(k=10, maxIter=10)
model = lda.fit(dataset)

ll = model.logLikelihood(dataset)
lp = model.logPerplexity(dataset)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound bound on perplexity: " + str(lp))

# Describe topics.
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

# Shows the result
transformed = model.transform(dataset)
transformed.show(truncate=False)

The lower bound on the log likelihood of the entire corpus: -827.8003455155235
The upper bound bound on perplexity: 3.1838474827717205
The topics described by their top-weighted terms:
+-----+-----------+---------------------------------------------------------------+
|topic|termIndices|termWeights                                                    |
+-----+-----------+---------------------------------------------------------------+
|0    |[4, 9, 10] |[0.10158301609032006, 0.0985092726875857, 0.09521609035428086] |
|1    |[10, 3, 4] |[0.10090365724969319, 0.09968453460929846, 0.09567470319271196]|
|2    |[4, 7, 0]  |[0.1046729147244217, 0.10438472358321176, 0.09227264111813155] |
|3    |[9, 10, 7] |[0.10560355145611457, 0.10158223531376381, 0.09847864083615746]|
|4    |[8, 6, 7]  |[0.11115617107599804, 0.10041443642845718, 0.09751311909343036]|
|5    |[6, 2, 7]  |[0.10761969057162588, 0.10071082117937362, 0.09690971048659572]|
|6    |[9, 3, 5]  |[0.10938217409826899, 0.1018404972067511

In [9]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

# Load and parse the data
data = sc.textFile("/usr/local/spark/data/mllib/sample_lda_data.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

# Save and load model
ldaModel.save(sc, "target/org/apache/spark/PythonLatentDirichletAllocationExample/LDAModel")
sameModel = LDAModel\
    .load(sc, "target/org/apache/spark/PythonLatentDirichletAllocationExample/LDAModel")

Learned topics (as distributions over vocab of 11 words):
Topic 0:
 4.30628899514
 8.77670552607
 3.32370995718
 4.34493638902
 1.05512825679
 1.42190848757
 20.0643902159
 0.480908902754
 5.95560059294
 9.73405621586
 29.2014870964
Topic 1:
 13.0397801356
 14.5728526484
 5.12046726951
 3.24974427807
 15.0091733854
 13.7862762935
 5.21751582821
 7.07033987903
 1.11384481395
 7.73801652649
 1.2324655316
Topic 2:
 8.65393086927
 5.65044182554
 3.55582277331
 32.4053193329
 8.93569835779
 6.79181521897
 5.71809395584
 2.44875121822
 0.930554593104
 6.52792725765
 2.56604737195


Py4JJavaError: An error occurred while calling o136.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/phejimlin/Documents/dcard_spark/target/org/apache/spark/PythonLatentDirichletAllocationExample/LDAModel/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1191)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1168)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:362)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1168)